# Introduction to atomman

- - -

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.

__Chandler A. Becker__, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), _Office of Data and Informatics, NIST_.

__Zachary T. Trautt__, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), _Materials Measurement Science Division, NIST_.

Version: 2017-02-24

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

This Notebook provides an introduction to creating and manipulating atomistic systems using atomman. It provides a working demonstration of how the code works. Links are also given to the in-depth Notebooks related to the specific actions mentioned here.

- - -

Library Imports

In [1]:
#Standard libraries
from __future__ import print_function
import os
from copy import deepcopy

#http://www.numpy.org/
import numpy as np    

#https://github.com/usnistgov/atomman
import atomman as am        
import atomman.unitconvert as uc

## 0. Unit conversions

Unit conversions are handled by the [atomman.unitconvert](../reference/atomman.unitconvert.ipynb) module. By default, atomman assumes that the working units of values are consistent with:

- length in Angstroms

- mass in atomic mass unit

- energy in electron volts

- charge in electron charge

- temperature in Kelvin

[set_in_units](atomman.unitconvert.set_in_units.ipynb) converts values from the specified units to the working units

In [2]:
five_feet = uc.set_in_units(5, 'foot')
print(five_feet, 'Angstroms')

15240000000.0 Angstroms


[get_in_units](atomman.unitconvert.get_in_units.ipynb) converts values from the working units to the specified units

In [3]:
print(uc.get_in_units(five_feet, 'inch'), 'inches')

60.0 inches


Complex unit expressions can be used, for instance 

    pg/(um*us^2) = 1000 Pa = kPa

In [4]:
#Set in complex units
pressure = uc.set_in_units(5, 'pg / (um * us^2)')

#Print in kPa
print(uc.get_in_units(pressure, 'kPa'), 'kPa')

5.0 kPa


## 1. Generate a System

An [atomman.System](../reference/atomman.System.ipynb) can be generated from scratch by combining an [atomman.Atoms](../reference/atomman.Atoms.ipynb) object and an [atomman.Box](../reference/atomman.Box.ipynb) object.

### 1.1 Create an Atoms object

The [atomman.Atoms](../reference/atomman.Atoms.ipynb) class contains the per-atom properties for all of the atoms. 

By default, each Atoms instance has two per-atom properties: 

- __atype__ is an integer atomic type

- __pos__ is the 3D vector position.  

Other properties can be freely assigned to all atoms at any time with any name. 

Create an Atoms object consistent with a simple face-centered cell and all atoms of the same type

In [5]:
atype = [1,1,1,1]
pos = [[0.0, 0.0, 0.0], 
       [0.5, 0.5, 0.0], 
       [0.5, 0.0, 0.5], 
       [0.0, 0.5, 0.5]]

atoms = am.Atoms(natoms=4, prop={'atype':atype, 'pos':pos})
print(atoms)

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.500 |   0.500 |   0.000
      2 |       1 |   0.500 |   0.000 |   0.500
      3 |       1 |   0.000 |   0.500 |   0.500


### 1.2 Create a Box object

The [atomman.Box](../reference/atomman.Box.ipynb) class represents a generic parallelopid in space as four vectors: the vectors for the three independent box directions, and an origin position vector.  It can be created using lattice parameters, LAMMPS box terms, or by explicitly giving the underlying vectors. 

Create a cubic box

In [6]:
box = am.Box(a=4.05, b=4.05, c=4.05)
print(box)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]


### 1.3 Create a System by combining Atoms and Box

A [atomman.System](../reference/atomman.System.ipynb) is then constructed by combining the [atomman.Atoms](../reference/atomman.Atoms.ipynb) and [atomman.Box](../reference/atomman.Box.ipynb).  The term [pbc](../reference/atomman.pbc.ipynb) is a tuple of three Boolean values indicating which directions are periodic (True). The argument "scale=True" scales the Atoms' pos values according to the Box definition. 

In [7]:
system = am.System(atoms=atoms, box=box, pbc=(True, True, True), scale=True)
print(system)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


### 1.4. Load a System from a file

The [atomman.load](../reference/atomman.load.ipynb) function generates a system by reading in data from an external format. The load function supports

- CIF files
- POSCAR files
- LAMMPS data and dump files
- json/xml atomic-system data models
- ase.Atoms objects
- pymatgen.Structure objects 

Create a LAMMPS dump file "fcc.dump"

In [8]:
with open('fcc.dump', 'w') as f:
    f.write("""
ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
4
ITEM: BOX BOUNDS pp pp pp
0.000000 4.050000
0.000000 4.050000
0.000000 4.050000
ITEM: ATOMS id type x y z
1 1 0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
2 1 2.0250000000000e+00 2.0250000000000e+00 0.0000000000000e+00
3 1 2.0250000000000e+00 0.0000000000000e+00 2.0250000000000e+00
4 1 0.0000000000000e+00 2.0250000000000e+00 2.0250000000000e+00""")

Load from "fcc.dump" and show the system info

In [9]:
new_system, symbols = am.load('atom_dump', 'fcc.dump')
print(new_system)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


## 2. System Manipulations

Atomman has a number of built-in System manipulation tools.

### 2.1 Rotating a System

The [atomman.rotate](../rotate/atomman.rotate.ipynb) and [atomman.rotate_cubic](../rotate/atomman.rotate_cubic.ipynb) functions allow for a system to be rotated using a 3x3 array of right-handed orthogonal vectors. 

Define axes for rotation (cubic crystallographic axes directions)

In [10]:
axes = np.array([[ 1, 1, 0],
                 [-1, 1, 0], 
                 [ 0, 0, 1]])

[atomman.rotate](../rotate/atomman.rotate.ipynb) returns a new System with Box vectors and atom positions transformed by axes. This transformation does not alter the box in any way.

In [11]:
print(am.rotate(system, axes))

avect =  [ 2.864, -2.864,  0.000]
bvect =  [ 2.864,  2.864,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.864 |   0.000 |   0.000
      2 |       1 |   1.432 |  -1.432 |   2.025
      3 |       1 |   1.432 |   1.432 |   2.025


[atomman.rotate_cubic](../rotate/atomman.rotate_cubic.ipynb) returns a new orthogonal System by rotating a cubic system and expanding the Box size to ensure atoms remain consistent across periodic boundaries.

In [12]:
print(am.rotate_cubic(system, axes))

avect =  [ 5.728,  0.000,  0.000]
bvect =  [ 0.000,  5.728,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 256
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.125 |   0.125 |   0.000
      2 |       1 |   0.125 |   0.000 |   0.125
      3 |       1 |   0.000 |   0.125 |   0.125
      4 |       1 |   0.250 |   0.000 |   0.000
      5 |       1 |   0.375 |   0.125 |   0.000
      6 |       1 |   0.375 |   0.000 |   0.125
      7 |       1 |   0.250 |   0.125 |   0.125
      8 |       1 |   0.500 |   0.000 |   0.000
      9 |       1 |   0.625 |   0.125 |   0.000
     10 |       1 |   0.625 |   0.000 |   0.125
     11 |       1 |   0.500 |   0.125 |   0.125
     12 |       1 |   0.750 |   0.000 |   0.000
     13 |       1 |   0.875 |   0.125 |   0.000
     14 |       1 |   0.875 |   0.000 |   0.125
     15 |       1 |   0.750 |   0.125 |   0.125
     16 |       1 |   0

### 2.2. Multiplying the size of a System

A larger System (i.e. supercell) can be generated using the [atomman.supersize](../reference/atomman.supersize.ipynb) function or the [atomman.System.supersize](../reference/atomman.System.supersize.ipynb) method.

Make system a 2x2x2 supercell of itself using [atomman.System.supersize](../reference/atomman.System.supersize.ipynb)

In [13]:
system.supersize(2, 2, 2)
print(system)

avect =  [ 8.100,  0.000,  0.000]
bvect =  [ 0.000,  8.100,  0.000]
cvect =  [ 0.000,  0.000,  8.100]
origin = [ 0.000,  0.000,  0.000]
natoms = 32
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025
      4 |       1 |   4.050 |   0.000 |   0.000
      5 |       1 |   6.075 |   2.025 |   0.000
      6 |       1 |   6.075 |   0.000 |   2.025
      7 |       1 |   4.050 |   2.025 |   2.025
      8 |       1 |   0.000 |   4.050 |   0.000
      9 |       1 |   2.025 |   6.075 |   0.000
     10 |       1 |   2.025 |   4.050 |   2.025
     11 |       1 |   0.000 |   6.075 |   2.025
     12 |       1 |   4.050 |   4.050 |   0.000
     13 |       1 |   6.075 |   6.075 |   0.000
     14 |       1 |   6.075 |   4.050 |   2.025
     15 |       1 |   4.050 |   6.075 |   2.025
     16 |       1 |   0.

### 2.3 System family conversions

The [atomman.trig_to_hex](../reference/atomman.trig_to_hex.ipynb) and [atomman.hex_to_ortho](../reference/atomman.hex_to_ortho.ipynb) functions provide a means of converting standard trigonal cells to hexagonal cells and standard hexagonal cells to orthogonal cells. 

### 2.4 Defect generation

The [atomman.defect](../reference/atomman.defect.ipynb) module provides tools for generating and analysing crystalline defects.

## 3. Basic Analysis Tools

### 3.1 Calculating vectors between atoms

The vector distances between atoms can be computed using the [atomman.dvect](../reference/atomman.dvect.ipynb) function, the [atomman.System.dvect](../reference/atomman.System.dvect.ipynb) method and the [atomman.displacement](../reference/atomman.displacement.ipynb) function.

[atomman.System.dvect](../reference/atomman.System.dvect.ipynb) identifies the shortest vector distance between two points/atom positions within a System.

Print the vector between atoms with ids 1 and 5

In [14]:
print(system.dvect(1,5))

[ 4.05  0.    0.  ]


Print the vectors between all atoms and position [0.1, 0.1, 0.1]

In [15]:
pos = np.array([0.1, 0.1, 0.1])
a_ids = range(system.natoms)

print(system.dvect(pos, a_ids))

[[-0.1   -0.1   -0.1  ]
 [ 1.925  1.925 -0.1  ]
 [ 1.925 -0.1    1.925]
 [-0.1    1.925  1.925]
 [ 3.95  -0.1   -0.1  ]
 [-2.125  1.925 -0.1  ]
 [-2.125 -0.1    1.925]
 [ 3.95   1.925  1.925]
 [-0.1    3.95  -0.1  ]
 [ 1.925 -2.125 -0.1  ]
 [ 1.925  3.95   1.925]
 [-0.1   -2.125  1.925]
 [ 3.95   3.95  -0.1  ]
 [-2.125 -2.125 -0.1  ]
 [-2.125  3.95   1.925]
 [ 3.95  -2.125  1.925]
 [-0.1   -0.1    3.95 ]
 [ 1.925  1.925  3.95 ]
 [ 1.925 -0.1   -2.125]
 [-0.1    1.925 -2.125]
 [ 3.95  -0.1    3.95 ]
 [-2.125  1.925  3.95 ]
 [-2.125 -0.1   -2.125]
 [ 3.95   1.925 -2.125]
 [-0.1    3.95   3.95 ]
 [ 1.925 -2.125  3.95 ]
 [ 1.925  3.95  -2.125]
 [-0.1   -2.125 -2.125]
 [ 3.95   3.95   3.95 ]
 [-2.125 -2.125  3.95 ]
 [-2.125  3.95  -2.125]
 [ 3.95  -2.125 -2.125]]


[atomman.displacement](../reference/atomman.displacement.ipynb) calculates the shortest vector distance between all corresponding atoms in two different Systems.

Define system_2 with positions shifted by [0.5, 0.5, 0.5] from system's positions

In [16]:
system_2 = deepcopy(system)
new_pos = system_2.atoms_prop(key='pos') + np.array([0.5,0.5,0.5])
system_2.atoms_prop(key='pos', value=new_pos)

Calculate the displacement of all atoms between the systems

In [17]:
print(am.displacement(system, system_2))

[[ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [ 0.5  0.5  0.5]]


### 3.2 Neighbor lists

The [atomman.NeighborList](../reference/atomman.NeighborList.ipynb) class and [atomman.System.neighbors](../reference/atomman.System.neighbors.ipynb) method compute neighbor lists for all of the atoms in the system within a specified cutoff. 

Build a neighbor list for all atoms in system

In [18]:
neighbors = am.NeighborList(system, cutoff=3)

The coordination number for each atom can be accessed with coord.

In [19]:
print('Average coordination =', neighbors.coord.mean())

Average coordination = 12.0


The list of neighbors for an atom can be retrieved by index

In [20]:
print(neighbors[0])

[ 1  2  3  5  6  9 11 13 18 19 22 27]


or iterated over

In [21]:
for nlist in neighbors:
    print(nlist)

[ 1  2  3  5  6  9 11 13 18 19 22 27]
[ 0  2  3  4  7  8 10 12 18 19 23 26]
[ 0  1  3  4  7  9 11 15 16 17 20 25]
[ 0  1  2  5  6  8 10 14 16 17 21 24]
[ 1  2  5  6  7  9 13 15 18 22 23 31]
[ 0  3  4  6  7  8 12 14 19 22 23 30]
[ 0  3  4  5  7 11 13 15 16 20 21 29]
[ 1  2  4  5  6 10 12 14 17 20 21 28]
[ 1  3  5  9 10 11 13 14 19 26 27 30]
[ 0  2  4  8 10 11 12 15 18 26 27 31]
[ 1  3  7  8  9 11 12 15 17 24 25 28]
[ 0  2  6  8  9 10 13 14 16 24 25 29]
[ 1  5  7  9 10 13 14 15 23 26 30 31]
[ 0  4  6  8 11 12 14 15 22 27 30 31]
[ 3  5  7  8 11 12 13 15 21 24 28 29]
[ 2  4  6  9 10 12 13 14 20 25 28 29]
[ 2  3  6 11 17 18 19 21 22 25 27 29]
[ 2  3  7 10 16 18 19 20 23 24 26 28]
[ 0  1  4  9 16 17 19 20 23 25 27 31]
[ 0  1  5  8 16 17 18 21 22 24 26 30]
[ 2  6  7 15 17 18 21 22 23 25 29 31]
[ 3  6  7 14 16 19 20 22 23 24 28 30]
[ 0  4  5 13 16 19 20 21 23 27 29 31]
[ 1  4  5 12 17 18 20 21 22 26 28 30]
[ 3 10 11 14 17 19 21 25 26 27 29 30]
[ 2 10 11 15 16 18 20 24 26 27 28 31]
[ 1  8  9 12

File cleanup

In [22]:
os.remove('fcc.dump')
os.remove('fcc.dump.json')

__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)